## Aruco
Aruco 是一种基于二进制平方标记的开源库，广泛用于计算机视觉中的标记检测和姿态估计。Aruco 标记是一种特殊的二维码，具有高对比度和独特的二进制模式，便于检测和识别。

### Aruco 标记的特点
1. **唯一性**：每个 Aruco 标记都有一个唯一的 ID，可以通过其二进制模式识别。
2. **高对比度**：标记通常是黑白的，便于在各种光照条件下检测。
3. **易于生成**：可以使用 Aruco 库轻松生成和打印标记。

### Aruco 的主要功能
1. **标记检测**：识别图像中的 Aruco 标记，并返回标记的 ID 和角点位置。
2. **姿态估计**：通过检测到的标记计算相机相对于标记的姿态（位置和方向）。
3. **标记绘制**：在图像上绘制检测到的标记，便于可视化。

### Aruco 库的使用
Aruco 库通常与 OpenCV 一起使用，以下是一些常见的操作：

1. **加载预定义字典**：
    ```csharp
    using var dictionary = CvAruco.GetPredefinedDictionary(PredefinedDictionaryName.Dict4X4_1000);
    ```
    预定义字典包含了一组预先生成的标记模式，可以直接使用。

2. **检测标记**：
    ```csharp
    CvAruco.DetectMarkers(src, dictionary, out var corners, out var ids, detectorParameters, out var rejectedPoints);
    ```
    该函数检测图像中的标记，并返回标记的角点、ID 和被拒绝的点。

3. **绘制检测到的标记**：
    ```csharp
    CvAruco.DrawDetectedMarkers(detectedMarkers, corners, ids, Scalar.Crimson);
    ```
    在图像上绘制检测到的标记，便于可视化。

4. **姿态估计**：
    ```csharp
    CvAruco.EstimatePoseSingleMarkers(corners, markerLength, cameraMatrix, distCoeffs, rvecs, tvecs);
    ```
    通过检测到的标记计算相机的姿态。

In [3]:
#r "nuget: OpenCvSharp4"
using OpenCvSharp;
using OpenCvSharp.Aruco;

Installed Packages OpenCvSharp4, 4.10.0.20240616

In [5]:
// The locations of the markers in the image at FilePath.Image.Aruco.
const int upperLeftMarkerId = 160;
const int upperRightMarkerId = 268;
const int lowerRightMarkerId = 176;
const int lowerLeftMarkerId = 168;

var src = Cv2.ImRead("assets/aruco_markers_photo.jpg");

var detectorParameters = new DetectorParameters();
detectorParameters.CornerRefinementMethod = CornerRefineMethod.Subpix;
detectorParameters.CornerRefinementWinSize = 9;

var dictionary = CvAruco.GetPredefinedDictionary(PredefinedDictionaryName.Dict4X4_1000);

CvAruco.DetectMarkers(src, dictionary, out var corners, out var ids, detectorParameters, out var rejectedPoints);

var detectedMarkers = src.Clone();
CvAruco.DrawDetectedMarkers(detectedMarkers, corners, ids, Scalar.Crimson);

// Find the index of the four markers in the ids array. We'll use this same index into the
// corners array to find the corners of each marker.
var upperLeftCornerIndex = Array.FindIndex(ids, id => id == upperLeftMarkerId);
var upperRightCornerIndex = Array.FindIndex(ids, id => id == upperRightMarkerId);
var lowerRightCornerIndex = Array.FindIndex(ids, id => id == lowerRightMarkerId);
var lowerLeftCornerIndex = Array.FindIndex(ids, id => id == lowerLeftMarkerId);

// Make sure we found all four markers.
if (upperLeftCornerIndex < 0 || upperRightCornerIndex < 0
        || lowerRightCornerIndex < 0 || lowerLeftCornerIndex < 0)
{
    return;
}

// Marker corners are stored clockwise beginning with the upper-left corner.
// Get the first (upper-left) corner of the upper-left marker.
var upperLeftPixel = corners[upperLeftCornerIndex][0];
// Get the second (upper-right) corner of the upper-right marker.
var upperRightPixel = corners[upperRightCornerIndex][1];
// Get the third (lower-right) corner of the lower-right marker.
var lowerRightPixel = corners[lowerRightCornerIndex][2];
// Get the fourth (lower-left) corner of the lower-left marker.
var lowerLeftPixel = corners[lowerLeftCornerIndex][3];

// Create coordinates for passing to GetPerspectiveTransform
var sourceCoordinates = new List<Point2f>
    {
        upperLeftPixel, upperRightPixel, lowerRightPixel, lowerLeftPixel
    };
var destinationCoordinates = new List<Point2f>
    {
        new Point2f(0, 0),
        new Point2f(1024, 0),
        new Point2f(1024, 1024),
        new Point2f(0, 1024),
    };

var transform = Cv2.GetPerspectiveTransform(sourceCoordinates, destinationCoordinates);
var normalizedImage = new Mat();
Cv2.WarpPerspective(src, normalizedImage, transform, new Size(1024, 1024));

Cv2.ImShow("Original Image", src);
Cv2.ImShow($"Found {ids.Length} Markers", detectedMarkers);
Cv2.ImShow("Normalized Image", normalizedImage);
Cv2.WaitKey();
Cv2.DestroyAllWindows();

Error: System.TypeInitializationException: The type initializer for 'OpenCvSharp.Internal.NativeMethods' threw an exception.
 ---> System.DllNotFoundException: Unable to load shared library 'OpenCvSharpExtern' or one of its dependencies. In order to help diagnose loading problems, consider using a tool like strace. If you're using glibc, consider setting the LD_DEBUG environment variable: 
/home/codespace/.nuget/packages/microsoft.dotnet-interactive/1.0.522904/tools/net8.0/any/runtimes/linux-x64/native/OpenCvSharpExtern.so: cannot open shared object file: No such file or directory
/usr/share/dotnet/shared/Microsoft.NETCore.App/8.0.7/OpenCvSharpExtern.so: cannot open shared object file: No such file or directory
/home/codespace/.nuget/packages/microsoft.dotnet-interactive/1.0.522904/tools/net8.0/any/runtimes/linux-x64/native/libOpenCvSharpExtern.so: cannot open shared object file: No such file or directory
/usr/share/dotnet/shared/Microsoft.NETCore.App/8.0.7/libOpenCvSharpExtern.so: can